In [1]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

In [2]:
llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

cache_dir = LocalFileStore(".cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter3.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = FAISS.from_documents(
    docs,
    cached_embeddings,
)

retriver = vectorstore.as_retriever()

C:\Users\김형준\AppData\Local\Temp\ipykernel_10908\935270013.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
C:\Users\김형준\AppData\Local\Temp\ipykernel_10908\935270013.py:18: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader("./files/chapter3.txt")
C:\Users\김형준\AppData\Local\Temp\ipykernel_10908\935270013.py:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package an

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 아주 도움이되는 비서입니다. 당신은 다음의 context만 사용해서 대답해야합니다. 만약 당신이 질문에 대한 답을 모른다면 절대로 추측하지 말고 반드시 모른다고 얘기하세요.\n\n{context}",
        ),
        MessagesPlaceholder(
            variable_name="history",
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

In [4]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = (
    (
        {
            "context": retriver,
            "question": RunnablePassthrough(),
        }
    )
    | RunnablePassthrough.assign(history=load_memory)
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

In [5]:
invoke_chain("Aaronson은 유죄인가요?")

죄를 지은 사람이 누구인지에 대한 구체적인 정보는 제가 가지고 있지 않습니다.


In [6]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

그가 쓴 메시지는 다음과 같습니다:
FREEDOM IS SLAVERY
TWO AND TWO MAKE FIVE
GOD IS POWER


In [7]:
invoke_chain("Julia 는 누구인가요?")

Julia는 위스톤과 함께 이야기 중에 나오는 캐릭터입니다. 그녀는 위스톤의 연인이자 동료입니다.


In [8]:
load_memory({})

[HumanMessage(content='Aaronson은 유죄인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='죄를 지은 사람이 누구인지에 대한 구체적인 정보는 제가 가지고 있지 않습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Julia 는 누구인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Julia는 위스톤과 함께 이야기 중에 나오는 캐릭터입니다. 그녀는 위스톤의 연인이자 동료입니다.', additional_kwargs={}, response_metadata={})]

In [9]:
invoke_chain("내가 처음에 한 질문은?")

당신이 처음에 한 질문은 "Aaronson은 유죄인가요?" 였습니다.


In [10]:
invoke_chain("Aaronson 는 누구인가요?")

Aaronson은 "1984"라는 소설에서 언급되는 캐릭터 중 하나입니다. 그는 이야기에서 중요한 역할을 하는 인물 중 하나입니다.


In [11]:
invoke_chain("Aaronson은 유죄인가요?")

죄를 지은 사람이 누구인지에 대한 구체적인 정보는 제가 가지고 있지 않습니다.


In [12]:
invoke_chain("마지막 질문은?")

마지막 질문은 "Aaronson은 유죄인가요?" 였습니다.
